In [27]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


In [28]:
df3= pd.read_csv('toronto.csv')
df3.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Then Let's focus on the specific borough "North York" in Toronto

In [30]:
NY = df3[df3['Borough'] == 'North York']
NY = NY.reset_index(drop=True)
NY.head(30)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
5,M2N,North York,Willowdale South,43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049
7,M2R,North York,Willowdale West,43.782736,-79.442259
8,M3A,North York,Parkwoods,43.753259,-79.329656
9,M3B,North York,Don Mills North,43.745906,-79.352188


#### Foursquare personal Infomation

In [31]:
CLIENT_ID = 'C4J035AMEDZE3JXWQPOKLGTZTMUGRECHXNAJ1VFWUNLTC4G4' #  Foursquare ID
CLIENT_SECRET = 'XKJC5AOLMNPIEKZW51SVXEZOTZ1X5CNRMVNCREKDUL4NUNGH' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C4J035AMEDZE3JXWQPOKLGTZTMUGRECHXNAJ1VFWUNLTC4G4
CLIENT_SECRET:XKJC5AOLMNPIEKZW51SVXEZOTZ1X5CNRMVNCREKDUL4NUNGH


#### Create a Map of North York and Its Neighbourhoods

In [32]:
address = 'North York,Toronto '
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_NY = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(NY['Latitude'], NY['Longitude'], NY['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY

In [39]:
LIMIT=100
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
NY_venues = getNearbyVenues(names=NY['Neighborhood'],
                                   latitudes=NY['Latitude'],
                                   longitudes=NY['Longitude']
                                  )

In [59]:
NY_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


In [60]:
Neighbor_list = list(NY_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside North York:')
print(len(Neighbor_list))
print('List of Neighborhoods inside North York:')
Neighbor_list

Number of Neighborhoods inside North York:
23
List of Neighborhoods inside North York:


['Hillcrest Village',
 'Fairview, Henry Farm, Oriole',
 'Bayview Village',
 'Silver Hills, York Mills',
 'Willowdale South',
 'York Mills West',
 'Willowdale West',
 'Parkwoods',
 'Don Mills North',
 'Flemingdon Park, Don Mills South',
 'Bathurst Manor, Downsview North, Wilson Heights',
 'Northwood Park, York University',
 'CFB Toronto, Downsview East',
 'Downsview West',
 'Downsview Central',
 'Downsview Northwest',
 'Victoria Village',
 'Bedford Park, Lawrence Manor East',
 'Lawrence Heights, Lawrence Manor',
 'Glencairn',
 'Maple Leaf Park, North Park, Upwood Park',
 'Humber Summit',
 'Emery, Humberlea']

In [61]:
Neighbor_venue_summary = NY_venues.groupby('Neighborhood').count()
Neighbor_venue_summary.head(20)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,4,4,4,4,4,4
"Emery, Humberlea",2,2,2,2,2,2


In [62]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(NY_venues['Venue Category'].unique())

There are 105 uniques categories.
Here is the list of different categories:


['Golf Course',
 'Pool',
 'Mediterranean Restaurant',
 'Dog Run',
 'Athletics & Sports',
 'Toy / Game Store',
 'Shopping Mall',
 'Bakery',
 'Tea Room',
 'Burger Joint',
 'Electronics Store',
 'Movie Theater',
 'Candy Store',
 'Pharmacy',
 'American Restaurant',
 'Fast Food Restaurant',
 'Coffee Shop',
 'Department Store',
 'Juice Bar',
 'Smoothie Shop',
 'Salon / Barbershop',
 'Clothing Store',
 'Bank',
 'Theater',
 'Food Court',
 'Restaurant',
 'Japanese Restaurant',
 'Cosmetics Shop',
 'Liquor Store',
 'Video Game Store',
 'Sporting Goods Shop',
 'Wings Joint',
 'Burrito Place',
 'Asian Restaurant',
 'Deli / Bodega',
 'Gift Shop',
 'Boutique',
 'Supplement Shop',
 "Women's Store",
 'Convenience Store',
 'Luggage Store',
 'Bus Station',
 'Metro Station',
 'Baseball Field',
 'Chinese Restaurant',
 'Café',
 'Cafeteria',
 'Grocery Store',
 'Steakhouse',
 'Ramen Restaurant',
 'Indonesian Restaurant',
 'Plaza',
 'Arts & Crafts Store',
 'Pet Store',
 'Sushi Restaurant',
 'Lounge',
 'Vietnam

In [63]:
NY_category= pd.get_dummies(data = NY_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
NY_category.head(50)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777803,-79.344226,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"Fairview, Henry Farm, Oriole",43.77

### Manually Selecting Related Features for the lunch bar

In [64]:
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'Golf Course',
 'Pool',
 'Dog Run',
 'Athletics & Sports',
 'Toy / Game Store',
 'Shopping Mall',
 'Electronics Store',
 'Movie Theater',
 'Candy Store',
 'Pharmacy',
 'Department Store',
 'Salon / Barbershop',
 'Clothing Store',
 'Bank',
 'Theater',
 'Restaurant',
 'Cosmetics Shop',
 'Liquor Store',
 'Video Game Store',
 'Sporting Goods Shop',
 'Gift Shop',
 'Boutique',
 'Supplement Shop',
 "Women's Store",
 'Convenience Store',
 'Luggage Store',
 'Bus Station',
 'Metro Station',
 'Baseball Field',
 'Grocery Store',
 'Plaza',
 'Arts & Crafts Store',
 'Pet Store',
 'Lounge',
 'Discount Store',
 'Hotel',
 'Park',
 'Gym / Fitness Center',
 'Gym',
 'General Entertainment',
 'Bike Shop',
 'Bridal Shop',
 'Video Store',
 'Massage Studio',
 'Furniture / Home Store',
 'Airport',
 'Bus Stop',
 'Moving Target',
 'Business Service',
 'Hockey Arena',
 'Intersection',
 'Hardware Store',
 'Accessories Store',
 'Event Space',
 'Miscellaneous Shop',
 'Playground',
 'Arcade',
 'Basketball Court',
 'Construction & Landscaping']

In [65]:
NY_hotspot = NY_category[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


NY_hotspot.head()

,Golf Course,Pool,Dog Run,Athletics & Sports,Toy / Game Store,Shopping Mall,Electronics Store,Movie Theater,Candy Store,Pharmacy,Department Store,Salon / Barbershop,Clothing Store,Bank,Theater,Restaurant,Cosmetics Shop,Liquor Store,Video Game Store,Sporting Goods Shop,Gift Shop,Boutique,Supplement Shop,Women's Store,Convenience Store,Luggage Store,Bus Station,Metro Station,Baseball Field,Grocery Store,Plaza,Arts & Crafts Store,Pet Store,Lounge,Discount Store,Hotel,Park,Gym / Fitness Center,Gym,General Entertainment,Bike Shop,Bridal Shop,Video Store,Massage Studio,Furniture / Home Store,Airport,Bus Stop,Moving Target,Business Service,Hockey Arena,Intersection,Hardware Store,Accessories Store,Event Space,Miscellaneous Shop,Playground,Arcade,Basketball Court,Construction & Landscaping
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bayview Village,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
Don Mills North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
feat_name_list = list(NY_hotspot.columns)
store_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Store') != (-1):
        store_list.append(value)
        
NY_hotspot['Total Stores'] = NY_hotspot[store_list].sum(axis = 1)
NY_hotspot = NY_hotspot.drop(columns = store_list)


feat_name_list = list(NY_hotspot.columns)
shop_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Shop') != (-1):
        shop_list.append(value)
        
NY_hotspot['Total Shops'] = NY_hotspot[shop_list].sum(axis = 1)
NY_hotspot = NY_hotspot.drop(columns = shop_list)

In [67]:
NY_hotspot.head(50)

,Golf Course,Pool,Dog Run,Athletics & Sports,Movie Theater,Pharmacy,Salon / Barbershop,Bank,Theater,Restaurant,Boutique,Bus Station,Metro Station,Baseball Field,Plaza,Lounge,Hotel,Park,Gym / Fitness Center,Gym,General Entertainment,Massage Studio,Airport,Bus Stop,Moving Target,Business Service,Hockey Arena,Intersection,Event Space,Playground,Arcade,Basketball Court,Construction & Landscaping,Total Stores,Total Shops
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2
Bayview Village,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Bedford Park, Lawrence Manor East",0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
Don Mills North,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Downsview Central,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
Downsview Northwest,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
Downsview West,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
"Emery, Humberlea",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### Showing the Fully-Processed DataFrame about Neighborhoods inside North York
##### This Dataset is Ready for any Machine Learning Algorithm.

In [68]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(NY_hotspot)

In [70]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = NY_hotspot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Golf Course,Pool,Dog Run,Athletics & Sports,Movie Theater,Pharmacy,Salon / Barbershop,Bank,Theater,Restaurant,Boutique,Bus Station,Metro Station,Baseball Field,Plaza,Lounge,Hotel,Park,Gym / Fitness Center,Gym,General Entertainment,Massage Studio,Airport,Bus Stop,Moving Target,Business Service,Hockey Arena,Intersection,Event Space,Playground,Arcade,Basketball Court,Construction & Landscaping,Total Stores,Total Shops,Total Sum
G2,0.0000,0.0000,0.0000,0.0000,1.000000e+00,1.0000,1.000000e+00,1.0000,1.000000e+00,2.0,1.000000e+00,1.000000e+00,1.000000e+00,1.0000,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.0000,0.000000e+00,0.000000e+00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.0000,0.0000,0.000,21.0000,5.0000,37.0000
G4,0.0000,0.0000,0.0000,0.0000,5.000000e-01,0.5000,0.000000e+00,0.0000,0.000000e+00,1.0,5.000000e-01,0.000000e+00,0.000000e+00,0.0000,5.000000e-01,5.000000e-01,5.000000e-01,0.0000,0.0000,0.000000e+00,0.000000e+00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,5.000000e-01,0.0000,0.0000,0.0000,0.000,6.0000,1.0000,11.5000
G3,0.0000,0.0000,0.0000,0.0000,0.000000e+00,0.5000,0.000000e+00,0.5000,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.0000,1.000000e+00,5.000000e-01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.0000,0.0000,0.000,2.5000,2.0000,8.0000
G5,0.0000,0.0000,0.0000,0.5000,0.000000e+00,0.5000,0.000000e+00,0.0000,0.000000e+00,0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,0.000000e+00,0.0000,0.5000,0.000000e+00,0.000000e+00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.0000,0.0000,0.000,3.0000,0.0000,5.0000
G1,0.0625,0.0625,0.0625,0.0625,2.775558e-17,0.0625,1.387779e-17,0.1875,1.387779e-17,0.0,2.775558e-17,1.387779e-17,1.387779e-17,0.1875,1.387779e-17,1.387779e-17,1.387779e-17,0.3125,0.0625,2.775558e-17,1.387779e-17,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,1.387779e-17,0.0625,0.0625,0.0625,0.125,0.1875,0.0625,2.0625


## Result:
#### Best Group is Group 2;

#### Finding the Corresponding Group for Each Neighborhood.

In [73]:
Neighborhood= pd.DataFrame([NY_hotspot.index, 1 + kmeans.labels_]).T
Neighborhood.columns = ['Neighborhood', 'Group']
Neighborhood

,Neighborhood,Group
0,"Bathurst Manor, Downsview North, Wilson Heights",3
1,Bayview Village,1
2,"Bedford Park, Lawrence Manor East",5
3,"CFB Toronto, Downsview East",1
4,Don Mills North,1
5,Downsview Central,1
6,Downsview Northwest,5
7,Downsview West,1
8,"Emery, Humberlea",1
9,"Fairview, Henry Farm, Oriole",2


In [74]:
Neighborhood[Neighborhood['Group'] == 2]

,Neighborhood,Group
9,"Fairview, Henry Farm, Oriole",2


Therefore, the best neighborhood to open a lunch bar are __Fairview, henry Farm or Oriole_